# **ConnectX Agent for Kaggle Environment**

In this notebook, we will create an agent for the **ConnectX** challenge using the **Kaggle Environments** API. This environment is used for simulating and testing agents in the **ConnectX** game, a variant of Connect Four.

### **Objectives**:
1. **Agent Creation**: Define the logic of the agent using Python functions.
2. **Agent Code Export**: Write the agent's code to a file.
3. **Testing the Agent**: Test the agent using the **Kaggle Environments**.

---

### **Step-by-Step Explanation of the Code**:

1. **Imports and Setup**: 
   - We will import necessary libraries, including `kaggle_environments` for environment setup, and `matplotlib.pyplot` for plotting.
  
2. **Write Agent to File**:
   - A function is created to export the agent's logic to a Python file. This is helpful for reusing the agent's code or sharing it for further evaluation.

3. **Environment Setup**:
   - We will create an environment for the **ConnectX** challenge, enabling debugging to visualize the agent's decisions and understand its actions during gameplay.

---

### **Next Steps**:
- After setting up the environment and preparing the agent, we will proceed to define the agent’s logic and integrate it into the environment for testing.


In [1]:
import numpy as np
import random
import inspect
import os
from kaggle_environments import make, evaluate
import matplotlib.pyplot as plt

# Function to write agent code to file
def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(f"{function.__name__} written to {file}")

# Create ConnectX environment for testing
env = make("connectx", debug=True)

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/registration.py", line 304, in register_gymnasium_envs
    _register_atari_envs()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/registration.py", line 205, in _register_atari_envs
    import ale_py
  File "/usr/local/lib/python3.11/dist-packages/ale_py/__init__.py", line 68, in <module>
    register_v0_v4_envs()
  File "/usr/local/lib/python3.11/dist-packages/ale_py/registration.py", line 178, in register_v0_v4_envs
    _register_rom_configs(legacy_games, obs_types, versions)
  File "/usr/local/lib/python3.11/dist-packages/ale_py/registration.py", line 63, in _register_rom_configs
    gymnasium.register(
    ^^^^^^^^^^^

# **ConnectX Agent - Dropping Pieces and Valid Moves**

In this section of the notebook, we define important functions that simulate the gameplay mechanics of **ConnectX** (a variant of Connect Four).

### **Functions Overview**:

1. **`drop_piece()`**:
   - **Purpose**: This function simulates dropping a piece into the specified column of the grid and returns the updated grid with the newly dropped piece, along with the row where the piece was placed.
   - **Parameters**:
     - `grid`: The current state of the game grid.
     - `col`: The column number where the player intends to drop their piece.
     - `mark`: The player's mark (typically `1` or `2`).
     - `config`: The game configuration, including number of rows and columns.
   - **Returns**: The updated grid and the row where the piece was dropped, or `-1` if the column is full.

2. **`get_valid_moves()`**:
   - **Purpose**: This function returns a list of valid columns where a piece can be dropped. It checks for columns that are not full (i.e., the top row of the column contains a `0`).
   - **Parameters**:
     - `grid`: The current state of the game grid.
     - `config`: The game configuration, including number of rows and columns.
   - **Returns**: A list of columns that are not full.

### **Next Steps**:
- These functions will be used in the agent's decision-making process to select valid moves and simulate the game as the agent plays.


In [2]:
def drop_piece(grid, col, mark, config):
    """Drop a piece in the specified column and return the new grid."""
    next_grid = grid.copy()
    for row in range(config.rows - 1, -1, -1):
        if next_grid[row][col] == 0:
            next_grid[row][col] = mark
            return next_grid, row
    return next_grid, -1  # Column full

def get_valid_moves(grid, config):
    """Return list of valid columns (not full)."""
    return [c for c in range(config.columns) if grid[0][c] == 0]

# **ConnectX Agent - Heuristic Evaluation Functions**

In this section, we define several functions that calculate a **heuristic score** for the **ConnectX** game. The heuristic score helps the agent evaluate different board states and make decisions during gameplay.

### **Functions Overview**:

1. **`check_window()`**:
   - **Purpose**: This function checks if a given **window** (a set of consecutive cells) contains a specified number of discs (`num_discs`) of the same **piece** (either `1` or `2`) and enough empty spaces (`0`).
   - **Parameters**:
     - `window`: A list representing a set of consecutive cells.
     - `num_discs`: The number of discs the agent is looking for.
     - `piece`: The agent's piece (either `1` or `2`).
     - `config`: The game configuration containing the size of the window (`inarow`).
   - **Returns**: `True` if the window contains `num_discs` of the same piece and the necessary number of empty spaces, otherwise `False`.

2. **`count_windows()`**:
   - **Purpose**: This function counts how many "windows" (a set of consecutive cells) contain a specified number of discs (`num_discs`) for the agent’s piece. It checks horizontal, vertical, and diagonal windows.
   - **Parameters**:
     - `grid`: The current game grid.
     - `num_discs`: The number of discs the agent is looking for.
     - `piece`: The agent's piece (either `1` or `2`).
     - `config`: The game configuration.
   - **Returns**: The total number of windows that match the criteria for the specified piece.

3. **`get_heuristic()`**:
   - **Purpose**: This function calculates a **heuristic score** for the current board configuration. The score is based on the number of windows with two, three, and four discs, both for the agent’s pieces and the opponent's pieces. The heuristic also includes a **center column bonus** to encourage placing pieces in the middle column.
   - **Parameters**:
     - `grid`: The current game grid.
     - `mark`: The agent's piece (either `1` or `2`).
     - `config`: The game configuration.
   - **Returns**: The calculated heuristic score for the current board state.

---

### **Score Weights**:
- **`A`**: Weight for winning (4 in a row) - `1e6`
- **`B`**: Weight for three in a row - `100`
- **`C`**: Weight for two in a row - `10`
- **`D`**: Penalty for opponent’s two in a row - `-50`
- **`E`**: Penalty for opponent’s three in a row - `-1000`

The heuristic function is designed to evaluate how favorable the board is for the agent, considering the number of potential winning positions and the positions of the opponent’s pieces.

---

### **Next Steps**:
- These functions will be used by the agent to evaluate the game state and make decisions to place pieces strategically on the board.


In [3]:
def check_window(window, num_discs, piece, config):
    """Check if a window has num_discs of piece and enough empty spaces."""
    return window.count(piece) == num_discs and window.count(0) == config.inarow - num_discs

def count_windows(grid, num_discs, piece, config):
    """Count windows with num_discs of piece in horizontal, vertical, and diagonal directions."""
    num_windows = 0
    # Horizontal
    for row in range(config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[row, col:col + config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # Vertical
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.columns):
            window = list(grid[row:row + config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # Positive diagonal
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[range(row, row + config.inarow), range(col, col + config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # Negative diagonal
    for row in range(config.inarow - 1, config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

def get_heuristic(grid, mark, config):
    """Calculate heuristic score for the board."""
    # Score weights
    A = 1e6  # Win (4 in a row)
    B = 100   # Three in a row
    C = 10    # Two in a row
    D = -50   # Opponent two in a row
    E = -1000 # Opponent three in a row
    
    num_twos = count_windows(grid, 2, mark, config)
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_twos_opp = count_windows(grid, 2, 3 - mark, config)
    num_threes_opp = count_windows(grid, 3, 3 - mark, config)
    
    # Center column bonus (encourage central play)
    center_col = config.columns // 2
    center_count = sum(1 for row in range(config.rows) if grid[row, center_col] == mark)
    center_bonus = center_count * 5
    
    score = A * num_fours + B * num_threes + C * num_twos + D * num_twos_opp + E * num_threes_opp + center_bonus
    return score

In [4]:
def is_terminal_window(window, config):
    """Check if a window contains a winning sequence."""
    return window.count(1) == config.inarow or window.count(2) == config.inarow

def is_terminal_node(grid, config):
    """Check if the game has ended (win or draw)."""
    # Check for draw (no empty cells in top row)
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win in all directions
    # Horizontal
    for row in range(config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[row, col:col + config.inarow])
            if is_terminal_window(window, config):
                return True
    # Vertical
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.columns):
            window = list(grid[row:row + config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # Positive diagonal
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[range(row, row + config.inarow), range(col, col + config.inarow)])
            if is_terminal_window(window, config):
                return True
    # Negative diagonal
    for row in range(config.inarow - 1, config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

# **Minimax with Alpha-Beta Pruning for ConnectX**

In this section, we define the **Minimax** algorithm with **alpha-beta pruning** to evaluate the board state and determine the best move for the agent.

### **Minimax Algorithm**:
The **Minimax** algorithm is a decision-making algorithm used in game theory. It evaluates the possible moves of the agent (maximizing player) and the opponent (minimizing player) to determine the best move based on the **heuristic evaluation** of the board.

### **Alpha-Beta Pruning**:
**Alpha-beta pruning** is an optimization technique used to improve the performance of the **Minimax** algorithm. It avoids evaluating branches of the game tree that cannot possibly influence the final decision. By maintaining two values, **alpha** and **beta**, the algorithm can "prune" (skip) certain branches of the tree.

### **Function Overview**:

1. **`minimax(grid, depth, alpha, beta, maximizingPlayer, mark, config)`**:
   - **Purpose**: This function implements the **Minimax** algorithm with **alpha-beta pruning** to evaluate the best possible move for the agent. It recursively evaluates the game tree to a given **depth** and returns a value representing the best move.
   - **Parameters**:
     - `grid`: The current state of the game board.
     - `depth`: The maximum depth to search in the game tree.
     - `alpha`: The best value the maximizing player can guarantee so far.
     - `beta`: The best value the minimizing player can guarantee so far.
     - `maximizingPlayer`: Boolean indicating whether it's the maximizing player's turn (True for agent, False for opponent).
     - `mark`: The agent's piece (either `1` or `2`).
     - `config`: The game configuration containing the number of rows, columns, and `inarow` for winning.
   - **Returns**: A score representing the best possible outcome (win, loss, or draw) for the agent.

2. **Terminal Node Check**:
   - **`is_terminal_node(grid, config)`**: A helper function to check if the current board state is a terminal state (win, loss, or draw).

3. **Heuristic Evaluation**:
   - **`get_heuristic(grid, mark, config)`**: This function evaluates the current board state based on the number of windows with two, three, and four consecutive pieces, as well as the positions of the opponent's pieces.

### **Minimax with Alpha-Beta Pruning**:

- The algorithm evaluates the game tree recursively by alternating between the maximizing player (the agent) and the minimizing player (the opponent).
- **Maximizing player**: The agent aims to maximize the score (win).
- **Minimizing player**: The opponent aims to minimize the score (prevent agent from winning).



    

In [5]:
def minimax(grid, depth, alpha, beta, maximizingPlayer, mark, config):
    """Minimax with alpha-beta pruning to evaluate board state."""
    if is_terminal_node(grid, config):
        if count_windows(grid, config.inarow, mark, config) > 0:
            return 1e6  # Win
        elif count_windows(grid, config.inarow, 3 - mark, config) > 0:
            return -1e6  # Loss
        return 0  # Draw
    if depth == 0:
        return get_heuristic(grid, mark, config)
    
    valid_moves = get_valid_moves(grid, config)
    if not valid_moves:
        return 0  # Draw
    
    if maximizingPlayer:
        value = -np.inf
        for col in valid_moves:
            next_grid, row = drop_piece(grid, col, mark, config)
            if row != -1:
                value = max(value, minimax(next_grid, depth - 1, alpha, beta, False, mark, config))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
        return value
    else:
        value = np.inf
        for col in valid_moves:
            next_grid, row = drop_piece(grid, col, 3 - mark, config)
            if row != -1:
                value = min(value, minimax(next_grid, depth - 1, alpha, beta, True, mark, config))
                beta = min(beta, value)
                if alpha >= beta:
                    break
        return value

# **ConnectX Agent - Minimax with Heuristics**

In this section, we define an **agent** for the **ConnectX** game using the **Minimax** algorithm with heuristic evaluation to make optimal decisions during gameplay.

### **Agent Overview**:
The agent uses the **Minimax** algorithm with **alpha-beta pruning** and a **heuristic evaluation** function to determine the best possible move. The agent will:

1. **Immediate Win Check**: First, it checks if there is a move that will result in an immediate win.
2. **Immediate Block Check**: If no immediate win is found, it checks if there is a move that will block the opponent from winning.
3. **Minimax Evaluation**: If neither an immediate win nor a block is found, the agent uses the **Minimax** algorithm to evaluate the possible moves and selects the one with the highest score.
4. **Random Selection**: If there are multiple moves with the same score, it randomly chooses one of them.

### **Steps Involved**:

1. **Convert Board to 2D Grid**:
   - The board is represented as a 1D list of cells. We reshape it into a 2D grid that matches the game’s configuration.

2. **Immediate Win Check**:
   - For each valid move, the agent simulates the move and checks if it results in a win for the agent. If a winning move is found, the agent makes that move immediately.

3. **Immediate Block Check**:
   - If no immediate winning move is found, the agent checks for any moves that would block the opponent from winning.

4. **Minimax with Heuristic**:
   - If no immediate win or block is found, the agent evaluates all valid moves using the **Minimax** algorithm with a depth of 3. The **Minimax** function uses **alpha-beta pruning** to optimize the search.

5. **Move Selection**:
   - The agent selects the move with the highest score. If multiple moves have the same score, the agent chooses one randomly.

### **Heuristic Evaluation**:
- The agent uses a heuristic function to evaluate each board state based on the number of consecutive pieces (two, three, and four in a row) and the positions of the opponent's pieces.

### **Next Steps**:
- This agent will be used to compete in the **ConnectX** environment, using the strategies defined above to make decisions in a dynamic gameplay environment.


In [6]:
def score_move(grid, col, mark, config, nsteps):
    """Score a move using minimax."""
    next_grid, row = drop_piece(grid, col, mark, config)
    if row == -1:
        return -np.inf
    return minimax(next_grid, nsteps - 1, -np.inf, np.inf, False, mark, config)

In [7]:
def agent(obs, config):
    """ConnectX agent with minimax and heuristics."""
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    valid_moves = get_valid_moves(grid, config)
    if not valid_moves:
        return 0
    # Immediate win
    for col in valid_moves:
        next_grid, row = drop_piece(grid, col, obs.mark, config)
        if row != -1 and count_windows(next_grid, config.inarow, obs.mark, config) > 0:
            return col
    # Immediate block
    for col in valid_moves:
        next_grid, row = drop_piece(grid, col, 3 - obs.mark, config)
        if row != -1 and count_windows(next_grid, config.inarow, 3 - obs.mark, config) > 0:
            return col
    # Minimax
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, 3) for col in valid_moves]))
    max_cols = [col for col in scores if scores[col] == max(scores.values())]
    return random.choice(max_cols)

In [8]:
# Write complete submission.py
with open("submission.py", "w") as f:
    f.write('''
import numpy as np
import random

N_STEPS = 3
A = 1e6
B = 100
C = 10
D = -50
E = -1000

def agent(obs, config):
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows - 1, -1, -1):
            if next_grid[row][col] == 0:
                next_grid[row][col] = mark
                return next_grid, row
        return next_grid, -1

    def get_valid_moves(grid, config):
        return [c for c in range(config.columns) if grid[0][c] == 0]

    def check_window(window, num_discs, piece, config):
        return window.count(piece) == num_discs and window.count(0) == config.inarow - num_discs

    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        for row in range(config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[row, col:col + config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns):
                window = list(grid[row:row + config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[range(row, row + config.inarow), range(col, col + config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        for row in range(config.inarow - 1, config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows

    def get_heuristic(grid, mark, config):
        num_twos = count_windows(grid, 2, mark, config)
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_twos_opp = count_windows(grid, 2, 3 - mark, config)
        num_threes_opp = count_windows(grid, 3, 3 - mark, config)
        center_col = config.columns // 2
        center_count = sum(1 for row in range(config.rows) if grid[row, center_col] == mark)
        score = A * num_fours + B * num_threes + C * num_twos + D * num_twos_opp + E * num_threes_opp + center_count * 5
        return score

    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow

    def is_terminal_node(grid, config):
        if list(grid[0, :]).count(0) == 0:
            return True
        for row in range(config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[row, col:col + config.inarow])
                if is_terminal_window(window, config):
                    return True
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns):
                window = list(grid[row:row + config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        for row in range(config.rows - (config.inarow - 1)):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[range(row, row + config.inarow), range(col, col + config.inarow)])
                if is_terminal_window(window, config):
                    return True
        for row in range(config.inarow - 1, config.rows):
            for col in range(config.columns - (config.inarow - 1)):
                window = list(grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False

    def minimax(grid, depth, alpha, beta, maximizingPlayer, mark, config):
        if is_terminal_node(grid, config):
            if count_windows(grid, config.inarow, mark, config) > 0:
                return 1e6
            elif count_windows(grid, config.inarow, 3 - mark, config) > 0:
                return -1e6
            return 0
        if depth == 0:
            return get_heuristic(grid, mark, config)
        valid_moves = get_valid_moves(grid, config)
        if not valid_moves:
            return 0
        if maximizingPlayer:
            value = -np.inf
            for col in valid_moves:
                next_grid, row = drop_piece(grid, col, mark, config)
                if row != -1:
                    value = max(value, minimax(next_grid, depth - 1, alpha, beta, False, mark, config))
                    alpha = max(alpha, value)
                    if alpha >= beta:
                        break
            return value
        else:
            value = np.inf
            for col in valid_moves:
                next_grid, row = drop_piece(grid, col, 3 - mark, config)
                if row != -1:
                    value = min(value, minimax(next_grid, depth - 1, alpha, beta, True, mark, config))
                    beta = min(beta, value)
                    if alpha >= beta:
                        break
            return value

    def score_move(grid, col, mark, config, nsteps):
        next_grid, row = drop_piece(grid, col, mark, config)
        if row == -1:
            return -np.inf
        return minimax(next_grid, nsteps - 1, -np.inf, np.inf, False, mark, config)

    try:
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        valid_moves = get_valid_moves(grid, config)
        if not valid_moves:
            return 0
        for col in valid_moves:
            next_grid, row = drop_piece(grid, col, obs.mark, config)
            if row != -1 and count_windows(next_grid, config.inarow, obs.mark, config) > 0:
                return col
        for col in valid_moves:
            next_grid, row = drop_piece(grid, col, 3 - obs.mark, config)
            if row != -1 and count_windows(next_grid, config.inarow, 3 - obs.mark, config) > 0:
                return col
        scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
        max_cols = [col for col in scores if scores[col] == max(scores.values())]
        return random.choice(max_cols)
    except Exception:
        return random.choice(valid_moves) if valid_moves else 0
    ''')
print("submission.py written with all dependencies")

submission.py written with all dependencies


# **ConnectX Agent - Environment Setup and Running the Agent**

In this section, we set up the **ConnectX** environment and run the agent against a random opponent. The environment is initialized, the agent plays against the opponent, and the game is rendered for visualization.

### **Steps Involved**:

1. **Reset the Environment**:
   - **`env.reset()`** is called to initialize the **ConnectX** environment. This method resets the environment to its initial state, preparing it for a new game. It returns the starting state of the game, including the board configuration and other necessary details.

2. **Run the Agent**:
   - **`env.run([agent, "random"])`**: This command runs the environment by playing a game between the **defined agent** and a **random opponent**. The agent plays its moves, and the random opponent makes random moves. The environment alternates between the two players until a win, draw, or timeout occurs.
   - The `agent` refers to the function that makes the decisions based on the Minimax algorithm and heuristics, while `"random"` refers to the random opponent.

3. **Render the Game**:
   - **`env.render(mode="ipython")`**: After the game is completed, the board and the gameplay are rendered in the **IPython** notebook using the `mode="ipython"` option. This renders the visual state of the game, showing the moves made by both the agent and the random opponent in the game.
   - The **`render()`** method is useful for debugging and visually understanding how the agent is performing during gameplay.

### **Next Steps**:
- After running the agent against the random opponent, we can further analyze the agent's performance, adjust strategies, or test it against other types of opponents.


In [9]:
env.reset()
env.run([agent, "random"])
env.render(mode="ipython")